<a href="https://colab.research.google.com/github/livjab/DS3-Kaggle-Comptetion/blob/master/Kaggle_challenge_(cleaned_up_notebook).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [20]:
test = pd.read_csv("https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/test_features.csv")
train = pd.read_csv("https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/train_features.csv")
train_y = pd.read_csv("https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/train_labels.csv")

train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
# fast first baseline

train_y["status_group"].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [21]:
# going to combine train_y with train for ease

train = pd.merge(train, train_y)
train.shape, test.shape

((59400, 41), (14358, 40))

In [7]:
train.isnull().sum().sort_values(ascending=False).head(10)

scheme_name          28166
scheme_management     3877
installer             3655
funder                3635
public_meeting        3334
permit                3056
subvillage             371
ward                     0
lga                      0
district_code            0
dtype: int64

In [0]:
def clean(df):
  
  # separating date into year/month/day
  df["date_recorded_sep"] = train["date_recorded"].str.split("-")
  
  # create new features for each
  df["year_recorded"] = pd.to_numeric(df["date_recorded_sep"].str[0])
  df["month_recorded"] = pd.to_numeric(df["date_recorded_sep"].str[1])
  df["day_of_month_recorded"] = pd.to_numeric(df["date_recorded_sep"].str[2])
  
  # drop separated column
  df = df.drop(columns="date_recorded_sep")
  
  
  # changing public_meeting to 0/1
  df["public_meeting"] = df.public_meeting.fillna(value="True")
  df["public_meeting"] = df.public_meeting.astype(bool).astype(int)
  
  # changing permit to 0/1
  df["permit"] = df.permit.fillna(value="True")
  df["permit"] = df.permit.astype(bool).astype(int)
  
  
  # drop the following features due to repetition and/or insignificance
  df = df.drop(columns=["id", # random number
                        "recorded_by", # same entry in every row
                        "extraction_type", # basically a duplicate
                        "extraction_type_group", # another near duplicate
                        "payment", # has a near duplicate
                        "water_quality", # has a near duplicate
                        "quantity_group", # has an exact duplicate
                        "source_type", # heas a near duplicate
                        "waterpoint_type_group"]) # has a near duplicate

  return df

In [22]:
train = clean(train)
train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,management_group,payment_type,quality_group,quantity,source,source_class,waterpoint_type,status_group,year_recorded,month_recorded,day_of_month_recorded
0,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,1,VWC,Roman,0,1999,gravity,vwc,user-group,annually,good,enough,spring,groundwater,communal standpipe,functional,2011,3,14
1,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,1,Other,NaN,1,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional,2013,3,6
2,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,1,VWC,Nyumba ya mungu pipe scheme,1,2009,gravity,vwc,user-group,per bucket,good,enough,dam,surface,communal standpipe multiple,functional,2013,2,25
3,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,1,VWC,NaN,1,1986,submersible,vwc,user-group,never pay,good,dry,machine dbh,groundwater,communal standpipe multiple,non functional,2013,1,28
4,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,1,NaN,NaN,1,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional,2011,7,13


In [23]:
test = clean(test)
test.shape, train.shape

((14358, 34), (59400, 35))

In [24]:
numeric_columns = train.select_dtypes(include="number").columns
numeric_columns.tolist()

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'public_meeting',
 'permit',
 'construction_year',
 'year_recorded',
 'month_recorded',
 'day_of_month_recorded']

In [25]:
categorical_columns = train.select_dtypes(exclude="number").columns
categorical_columns.tolist()

['date_recorded',
 'funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'scheme_management',
 'scheme_name',
 'extraction_type_class',
 'management',
 'management_group',
 'payment_type',
 'quality_group',
 'quantity',
 'source',
 'source_class',
 'waterpoint_type',
 'status_group']

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)])

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', XGBClassifier(max_depth=5))])

X = train.drop('status_group', axis=1)
y = train['status_group']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)
